In [1]:
# quick and dirty way to change the current working directory to root (/toxic-comment-classification)
# you should run this at least once just to be certain
import os
from os import chdir, path, getcwd
for i in range(10):
    if path.isfile("checkcwd"):
        break
    chdir(path.pardir)
if path.isfile("checkcwd"):
    pass
else:
    raise Exception("Something went wrong. cwd=" + getcwd())
root_path = os.getcwd()

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.linear_model import LogisticRegression
from gc import collect

from joblib import dump

In [3]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 2),
    max_features=10000,
    )

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(3, 6),
    max_features=40000,
    )

In [4]:
path = 'kaggle/input/'
comp = 'jigsaw-toxic-comment-classification-challenge/'
clean_data_path = 'clean_data/'
EMBEDDING_FILE=f'{path}glove_embeddings/glove.6B.300d.txt'
TRAIN_DATA_FILE=f'{path}{comp}train.csv.zip'
TEST_DATA_FILE=f'{path}{comp}test.csv.zip'
CLEAN_TRAIN_DATA_FILE=f'{clean_data_path}data_train_cleaned_light2.txt'
CLEAN_TEST_DATA_FILE=f'{clean_data_path}data_test_cleaned_light2.txt'
SAMPLE_SUBMISSION=f'{path}{comp}sample_submission.csv.zip'

In [5]:
save_path = 'model_checkpoint/logistic_regression/'

In [6]:
def read_from_file(filename):
    with open(filename, 'r') as f:
        return pd.Series(f.read().splitlines())

In [7]:
X_train = read_from_file(CLEAN_TRAIN_DATA_FILE)
X_test = read_from_file(CLEAN_TEST_DATA_FILE)

collect()

train_word_features = word_vectorizer.fit_transform(X_train)
dump(word_vectorizer, save_path + 'word_vectorizer.bin', compress=True)
train_char_features = char_vectorizer.fit_transform(X_train)
dump(char_vectorizer, save_path + 'char_vectorizer.bin', compress=True)

del X_train
collect()

X_t = sparse.hstack([train_word_features, train_char_features])

del train_word_features
del train_char_features
collect()

test_word_features = word_vectorizer.transform(X_test)
test_char_features = char_vectorizer.transform(X_test)

del X_test
collect()

X_te = sparse.hstack([test_word_features, test_char_features])
del test_word_features
del test_char_features
collect()

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [8]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)
y = train[list_classes].values
train_shape = train.shape[0]
test_shape = test.shape[0]

oof = pd.DataFrame.from_dict({'id': train['id']})
submid = pd.DataFrame({'id': test["id"]})


In [9]:
del train
del test
collect()

0

In [10]:
# from sklearn.model_selection import KFold
# import concurrent.futures
# num_folds = 10

# collect()

# scores = []
# oof_predict = np.zeros((train_shape, 6))

# predict = np.zeros((test_shape, 6))
# kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# lst_num = [0, 1, 2, 3, 4, 5]

# for train_index, test_index in kf.split(X_t):
    
#     kfold_y_train, kfold_y_test = y[train_index], y[test_index]
#     kfold_X_train = X_t[train_index]

#     kfold_X_valid = X_t[test_index]

    
#     def single_job(i: int):
#         # print('start')
#         model = LogisticRegression(C=2, penalty='l2' ,random_state=42, class_weight='balanced', solver='saga', max_iter=750)
#         model.fit(kfold_X_train, kfold_y_train[:, i])
#         return (i, model)
#         # return i, a, b
        
#     with concurrent.futures.ProcessPoolExecutor() as executor:
#         models = list(executor.map(single_job, lst_num))

#     models.sort(key=lambda x: x[0])

#     for model in models:
#         predict[:, model[0]] += model[1].predict_proba(X_te)[:, 1] / num_folds
#         oof_predict[test_index, model[0]] = model[1].predict_proba(kfold_X_valid)[:, 1]

#     print('fold done')

    
# print('Done')


# submission = pd.concat([submid, pd.DataFrame(predict, columns=list_classes)], axis=1)
# submission.to_csv('kaggle/working/9_sub.csv', index=False)


# for c in list_classes:   
#     oof[c] = np.zeros(train_shape)

    
# oof[list_classes] = oof_predict
# oof.to_csv('kaggle/working/9_oof.csv', index=False)

In [11]:
import concurrent.futures

collect()
predict = np.zeros((test_shape, 6))
lst_num = [0, 1, 2, 3, 4, 5]

def single_job(i: int):
    model = LogisticRegression(C=2, penalty='l2' ,random_state=42, class_weight='balanced', solver='saga', max_iter=750)
    model.fit(X_t, y[:, i])
    dump(model, save_path + 'logistic_regression_' + str(i) + '.bin', compress=True)
    return (i, model)

with concurrent.futures.ProcessPoolExecutor() as executor:
    models = list(executor.map(single_job, lst_num))

models.sort(key=lambda x: x[0])

for model in models:
    predict[:, model[0]] = model[1].predict_proba(X_te)[:, 1]


submission = pd.concat([submid, pd.DataFrame(predict, columns=list_classes)], axis=1)
submission.to_csv('kaggle/working/logistic_regression/1fold_logistic_regression.csv', index=False)

/home/thanh309/miniconda3/envs/thanh309-ml/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/thanh309/miniconda3/envs/thanh309-ml/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/thanh309/miniconda3/envs/thanh309-ml/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/thanh309/miniconda3/envs/thanh309-ml/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/thanh309/miniconda3/envs/thanh309-ml/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_